In [1]:
# nltk.download('stopwords')
# nltk.download('wordnet')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import string
import nltk
import it_core_news_sm
import re
from itertools import combinations
import itertools
import os
import networkx as nx

from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from wordcloud import WordCloud


In [2]:
database_words = pd.read_csv("~/Scrivania/common_start/database/id_lemmas.csv",index_col = 0 , sep=',', engine='python')
database_words['text_nlp'] = database_words.apply(lambda row: word_tokenize(row['text_nlp']), axis = 1)
database_words

,text_nlp
0,"[diro, gente, guerra, portato, casa, rimpatriare]"
1,"[marcello, perfavore]"
2,"[patrio, patriota, difeso, proprio, radice, so..."
3,"[musulmano, comandare, casa]"
4,"[odiare, dipendere, odiare, comandare, librare]"
...,...
78170,"[vedere, ruspa, inviato, salvini, ripulire, am..."
78171,"[fino, sentire, sempre, richiamare, africo, pa..."
78172,"[sparare, ladro, occhio, penna, barbetta]"
78173,"[consigliatissima, africo]"


In [3]:
database_comments = pd.read_csv("~/Scrivania/common_start/database/com_liwc.csv", sep='\t', engine='python')
database_comments.head(2)

,Origin_file_order,Site,p_id,dateCreated,p_politician,p_gender,p_GRUPPO_PE,p_LISTA,p_PARTITO,p_governo,...,p_Comma,p_Colon,p_SemiC,p_Qmark,p_Exclam,p_Dash,p_Quote,p_Apostro,p_Parenth,p_OtherP
0,30126,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30226,FB,96844400700_10157493758850701,2019-04-23T10:33:37Z,MARCELLO GEMMATO,M,PPE,FDI,FDI,opposizione,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# THE FOLLOWING CELLS ARE THE ONES TO BE MODIFIED IN ORDER TO OBTAIN ANY OTHER SUBSET

## Here keep only problematic and hate comments

In [4]:
subset_comments = database_comments[database_comments.loc[:,'c_rating3'] == 'probl-hate'].loc[:,'c_rating3']
#free memory
subset_comments

0        probl-hate
1        probl-hate
2        probl-hate
3        probl-hate
4        probl-hate
            ...    
78043    probl-hate
78044    probl-hate
78045    probl-hate
78046    probl-hate
78047    probl-hate
Name: c_rating3, Length: 6636, dtype: object

computes the maching indexes and retrieve the subset from the lemmas dataset

In [5]:
matching_indexes = database_words.index.intersection(subset_comments.index)
subset_words = database_words.loc[matching_indexes,:]
subset_words

,text_nlp
0,"[diro, gente, guerra, portato, casa, rimpatriare]"
1,"[marcello, perfavore]"
2,"[patrio, patriota, difeso, proprio, radice, so..."
3,"[musulmano, comandare, casa]"
4,"[odiare, dipendere, odiare, comandare, librare]"
...,...
78043,"[folla, quell, handicappare, linguacciuto, gio..."
78044,"[cinghiare, morono, terra]"
78045,"[farli, prendere, cinghiare, terra, fermo]"
78046,"[immigrato, rompere, coglione]"


In [6]:
##check for null values
subset_words.isnull().values.any()

False

In [7]:
##create a copy of the dataframe and add the index
df_words = subset_words.copy()
df_words['id'] = df_words.index
df_words

,text_nlp,id
0,"[diro, gente, guerra, portato, casa, rimpatriare]",0
1,"[marcello, perfavore]",1
2,"[patrio, patriota, difeso, proprio, radice, so...",2
3,"[musulmano, comandare, casa]",3
4,"[odiare, dipendere, odiare, comandare, librare]",4
...,...,...
78043,"[folla, quell, handicappare, linguacciuto, gio...",78043
78044,"[cinghiare, morono, terra]",78044
78045,"[farli, prendere, cinghiare, terra, fermo]",78045
78046,"[immigrato, rompere, coglione]",78046


Create a dataframe of unique words

In [8]:
unique_words = {}
words_per_comment = {}

for index, row in df_words.iterrows():
    if(len(row['text_nlp'])>0):
        words_per_comment[row["id"]]= [clean_txt for clean_txt in row['text_nlp']] ###iterating the column text_clean. There are 78174 rows
        for single_word in row['text_nlp']:
            unique_words.setdefault(single_word, 0) ## setdefault() method returns the value of a key (if the key is in dictionary)
            unique_words[single_word] += 1

#len of unique words
print("Number of unique words: ", len(unique_words))
unique_words

Number of unique words:  11008


{'diro': 125,
 'gente': 172,
 'guerra': 94,
 'portato': 18,
 'casa': 330,
 'rimpatriare': 20,
 'marcello': 3,
 'perfavore': 1,
 'patrio': 43,
 'patriota': 6,
 'difeso': 29,
 'proprio': 33,
 'radice': 13,
 'soccombere': 1,
 'roma': 36,
 'islamico': 90,
 'accoltellare': 3,
 'uomo': 54,
 'crocifisso': 9,
 'collare': 11,
 'accadere': 14,
 'stazione': 7,
 'termine': 12,
 'aggressore': 1,
 'fermare': 25,
 'poliziotto': 13,
 'notare': 12,
 'vittima': 16,
 'tentare': 7,
 'sgozzarlo': 1,
 'aumentare': 18,
 'controllo': 6,
 'attenzione': 17,
 'prevenire': 2,
 'violenza': 27,
 'cittadino': 47,
 'innocente': 11,
 'partire': 166,
 'imperativo': 1,
 'corrado': 4,
 'armeri': 2,
 'fdi': 7,
 'musulmano': 39,
 'comandare': 32,
 'odiare': 65,
 'dipendere': 12,
 'librare': 12,
 'dire': 207,
 'dovere': 142,
 'diventare': 43,
 'razzista': 72,
 'peggio': 38,
 'meditato': 8,
 'sinistronzi': 11,
 'tolleranza': 9,
 'rispettare': 76,
 'cultura': 54,
 'leggere': 141,
 'barcone': 24,
 'via': 142,
 'sottoscrivere':

In [9]:
folder_path = 'subsets/'
os.makedirs(folder_path,exist_ok=True)

## Create the unique words dataframe

Moreover as attribute it has how many times it appears in a given network, in this case hate comments

In [10]:
lemmas_df         = pd.DataFrame.from_dict(unique_words, orient='index').reset_index()
lemmas_df.columns = ['label', 'counts'] 
lemmas_df

,label,counts
0,diro,125
1,gente,172
2,guerra,94
3,portato,18
4,casa,330
...,...,...
11003,handicappare,1
11004,linguacciuto,1
11005,morono,1
11006,elettrizzare,1


In [11]:
lemmas_df = lemmas_df.reset_index()
lemmas_df.columns = ['id', 'label', 'counts']
lemmas_df

,id,label,counts
0,0,diro,125
1,1,gente,172
2,2,guerra,94
3,3,portato,18
4,4,casa,330
...,...,...,...
11003,11003,handicappare,1
11004,11004,linguacciuto,1
11005,11005,morono,1
11006,11006,elettrizzare,1


In [12]:
#export
lemmas_df.to_csv('subsets/NODES_hate_problematic_subset.csv', index = False)

In [13]:
def get_id_from_node(word, ids):
    return ids.index[ids.label == word][0]


In [14]:
words_per_comment = {}

for index, row in df_words.iterrows():
    if(len(row['text_nlp'])>0):
        words_per_comment[row["id"]]= [get_id_from_node(clean_txt, lemmas_df) for clean_txt in row['text_nlp']] ###iterating the column text_clean. There are 78174 rows

words_per_comment

{0: [0, 1, 2, 3, 4, 5],
 1: [6, 7],
 2: [8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  14,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  15,
  38,
  39,
  40,
  41],
 3: [42, 43, 4],
 4: [44, 45, 44, 43, 46],
 5: [47, 48, 49, 50, 51],
 6: [52, 53, 52],
 7: [54, 55, 56, 57, 58, 59],
 8: [60],
 9: [61, 62, 63, 64, 65, 48, 66, 67, 68, 69],
 10: [70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
 11: [39, 40, 88, 89, 90, 91, 92],
 12: [93, 94, 0, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 15],
 13: [107, 108, 109, 110, 111, 112],
 14: [113,
  114,
  115,
  116,
  117,
  80,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126],
 15: [127, 128],
 16: [129, 130, 131, 132, 133, 134, 135],
 17: [136, 137, 59, 122, 138, 139],
 18: [140, 127, 141, 142, 139, 143, 144, 145, 146, 147, 148],
 19: [149, 150, 150, 151, 152, 153],
 20: [154, 155],
 21: [156, 157],
 22: [15

In [15]:
##create a matrix of words
clean_df = pd.DataFrame(0, index=lemmas_df.index, columns=lemmas_df.index)

for key in words_per_comment:
    for pair in itertools.product(words_per_comment[key],words_per_comment[key]):
        if pair[0]!=pair[1] and not(clean_df.at[pair[0],pair[1]]):
            clean_df.at[pair[0],pair[1]] += 1
            clean_df.at[pair[1],pair[0]] += 1

display(clean_df) ##this is the full matrix of words

,0,1,2,3,4,5,6,7,8,9,...,10998,10999,11000,11001,11002,11003,11004,11005,11006,11007
0,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
3,1,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,0,1,0,0,1,0,...,0,0,1,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
11004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
11005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11006,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [16]:

##use this as a basis to get edges and their weights

words_projection = {} ##create dictionary 
##itertools.product() which computes the cartesian product of input iterables.

for key in words_per_comment:
    for pair in itertools.product(words_per_comment[key],words_per_comment[key]):
        if pair[0]!=pair[1] and not(pair[::-1] in words_projection):
            words_projection.setdefault(pair,0)
            words_projection[pair] += 1

words_projection

{(0, 1): 9,
 (0, 2): 6,
 (0, 3): 2,
 (0, 4): 13,
 (0, 5): 1,
 (1, 2): 2,
 (1, 3): 1,
 (1, 4): 20,
 (1, 5): 1,
 (2, 3): 6,
 (2, 4): 10,
 (2, 5): 2,
 (3, 4): 5,
 (3, 5): 1,
 (4, 5): 1,
 (6, 7): 1,
 (8, 9): 2,
 (8, 10): 1,
 (8, 11): 14,
 (8, 12): 1,
 (8, 13): 1,
 (8, 14): 4,
 (8, 15): 8,
 (8, 16): 1,
 (8, 17): 4,
 (8, 18): 1,
 (8, 19): 1,
 (8, 20): 5,
 (8, 21): 1,
 (8, 22): 1,
 (8, 23): 1,
 (8, 24): 1,
 (8, 25): 1,
 (8, 26): 1,
 (8, 27): 3,
 (8, 28): 1,
 (8, 29): 1,
 (8, 30): 1,
 (8, 31): 1,
 (8, 32): 1,
 (8, 33): 1,
 (8, 34): 1,
 (8, 35): 16,
 (8, 36): 1,
 (8, 37): 9,
 (8, 38): 1,
 (8, 39): 1,
 (8, 40): 1,
 (8, 41): 1,
 (9, 10): 1,
 (9, 11): 1,
 (9, 12): 1,
 (9, 13): 1,
 (9, 14): 2,
 (9, 15): 2,
 (9, 16): 1,
 (9, 17): 1,
 (9, 18): 1,
 (9, 19): 1,
 (9, 20): 1,
 (9, 21): 1,
 (9, 22): 1,
 (9, 23): 1,
 (9, 24): 1,
 (9, 25): 1,
 (9, 26): 1,
 (9, 27): 1,
 (9, 28): 1,
 (9, 29): 1,
 (9, 30): 1,
 (9, 31): 1,
 (9, 32): 1,
 (9, 33): 1,
 (9, 34): 1,
 (9, 35): 1,
 (9, 36): 1,
 (9, 37): 2,
 (9, 38): 1

In [17]:
##obtain weighted edges list
##edge lists =36million
#WEIGHTED
#to get weighted graph we need a list of 3-element tuplels (u,v,w) where u and v are nodes 
## and w is a number representing weight

words_weighted = []
for edge in words_projection:
    words_weighted.append((edge[0],edge[1],words_projection[edge]))

G = nx.Graph()
G.add_weighted_edges_from(words_weighted)
nx.write_weighted_edgelist(G, "subsets/EDGES_hate_problematic_subset.csv",delimiter=",") ##save the edges list as csv
df_edges = pd.read_csv('subsets/EDGES_hate_problematic_subset.csv')
df_edges.columns = ['source', 'target', 'weight']
df_edges.to_csv('subsets/EDGES_hate_problematic_subset.csv', index = False)

In [18]:
df_edges

,source,target,weight
0,0,2,6
1,0,3,2
2,0,4,13
3,0,5,1
4,0,93,1
...,...,...,...
379229,11000,11001,1
379230,11000,11002,1
379231,11001,11002,1
379232,11003,11004,1
